In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
import process
from PyFolioCC import PyFolioCC 

historical_data = df_cleaned
number_of_repetitions = 10
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.1
eta = 0.1

corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return_res = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return_res, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)
final_weights = process.final_weights(markowitz_weights=markowitz_weights_res, constituent_weights=constituent_weights)
W = process.training_phase(lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=number_of_clusters, sigma=sigma, evaluation_window=evaluation_window, eta=eta, clustering_method=clustering_method)

consolidated_W = process.consolidated_W(number_of_repetitions=number_of_repetitions, lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=number_of_clusters, sigma=sigma, evaluation_window=evaluation_window, eta=eta, clustering_method=clustering_method)

In [6]:
df_cleaned['AVT']


03/01/2000   -0.014660
04/01/2000   -0.039051
05/01/2000   -0.048625
06/01/2000   -0.011655
07/01/2000    0.011655
                ...   
24/12/2020   -0.009528
28/12/2020    0.010277
29/12/2020   -0.014516
30/12/2020   -0.000574
31/12/2020    0.012323
Name: AVT, Length: 5279, dtype: float64

In [3]:
portfolio_returns = process.portfolio_returns(evaluation_window=evaluation_window, df_cleaned=df_cleaned, lookback_window=lookback_window, consolidated_W=consolidated_W)
portfolio_returns

KeyError: 'AVT'

In [60]:
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_comp = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=number_of_clusters, lookback_window=lookback_window, clustering_method=clustering_method)
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_comp, sigma=sigma, lookback_window=lookback_window)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window)
markowitz_weights_res = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=lookback_window, evaluation_window=evaluation_window, eta=eta)
final_weights = process.final_weights(markowitz_weights=markowitz_weights_res, constituent_weights=constituent_weights)
print(corr_matrix.shape, len(cluster_comp), len(constituent_weights), len(cluster_return), len(markowitz_weights_res), len(final_weights))


(695, 695) 37 37 37 37 692


In [10]:
cluster_returns = pd.DataFrame(index = [f'cluster {i}' for i in range(1, len(constituent_weights) + 1)], columns = df_cleaned.columns[lookback_window[0]:lookback_window[1]])
for keys, value in constituent_weights['cluster 1'].items():
    print(keys, value)

AVA 3.921716041425027e-05
BKH 0.110883916406304
BXS 0.2035512296849513
DRQ 0.04082643926897566
INCY 1.0230500389790201e-11
LNT 0.1654632187161882
MTZ 0.14365464588183466
NI 0.05214005001251528
OKE 0.2394986472578898
PKI 0.01829458705579904
RMD 0.02564804854489745
